In [1]:
import os, sys, gc, subprocess  
os.chdir('../')
# print(sys.path)
# print(subprocess.check_output('pwd',shell=True, universal_newlines=True))

In [2]:
# import load_path
import config, data, model, util                                                
from datetime import datetime                                                   
from obspy.clients.fdsn import Client                                           
import obspy.core as oc                                                         

## 1. Configure DPP

In [3]:
# config
#
util.init_session()
dpp_config = config.Config()
dpp_config.set_trigger(pthres_p=[0.5, 0.001], pthres_s=[0.5, 0.001])
# dpp_config.set_trigger(pthres_p=[0.9, 0.001], pthres_s=[0.9, 0.001])
# dpp_config.set_trigger(pthres_p=[0.95, 0.001], pthres_s=[0.9, 0.001])
# dpp_config.set_picking(op_conds=['1','2','3','4'], tp_th_add=1.5, dt_sp_near=1.5, dt_ps_max=25., dt_sdup_max=2., mcd_iter=10, run_mcd=True)
dpp_config.set_picking(op_conds=['1','2','3','4'], tp_th_add=1.5, dt_sp_near=1.5, dt_ps_max=25., dt_sdup_max=2., run_mcd=False)
# dpp_config.set_picking(op_conds=[], tp_th_add=1.5, dt_sp_near=1.5, dt_ps_max=25., dt_sdup_max=2., mcd_iter=10, run_mcd=False)
dpp_config.set_data(
    stas=['PB01', 'PB02'],
    net='CX',
    ch='HH',
    archive='archive_CX_20140501_test',
    opath='out_CX_20140501_test'
)
dpp_config.set_time(
    dt_iter=3600.,
    #
    tstart = "2014-05-01T00:00:00",
    tend = "2014-05-01T12:00:00",
    #
#     tstart = "2014-06-01T12:00:00",
#     tend = "2014-06-02T00:00:00",     
)

__pycache__/ removed
~/.nv/ not found, continuing...


## 2. Download seismic data and read it into DPP

In [4]:
# download and archive seismic waveforms
#
client = Client("GFZ")
os.makedirs(f"{dpp_config.data['archive']}", exist_ok=True)
tstart = oc.UTCDateTime(dpp_config.time['tstart'])
tend = oc.UTCDateTime(dpp_config.time['tend'])
# tstart = oc.UTCDateTime(dpp_config.time['tstart']) - 500
# tend = oc.UTCDateTime(dpp_config.time['tstart']) + 600
for sta in dpp_config.data['stas']:
    st = client.get_waveforms(network=dpp_config.data['net'], station=sta, location="*", channel=f"{dpp_config.data['ch']}?", starttime=tstart, endtime=tend)
    # print(st)
    st_name = f"{dpp_config.data['archive']}/{st[0].stats.network}.{st[0].stats.station}..{st[0].stats.channel[:-1]}.mseed"
    print(f"writing stream {st_name}...")
    st.write(st_name, format="MSEED")

writing stream archive_CX_20140501_test/CX.PB01..HH.mseed...
writing stream archive_CX_20140501_test/CX.PB02..HH.mseed...


In [5]:
# data
#
dpp_data = data.Data()
dpp_data.read_from_directory(dpp_config)
#
# for k in dpp_data.data:
#     print(k, dpp_data.data[k])

reading seismic waveform: archive_CX_20140501_test/CX.PB01..HH.mseed
reading seismic waveform: archive_CX_20140501_test/CX.PB02..HH.mseed
6 Trace(s) in Stream:
CX.PB01..HHZ | 2014-04-30T23:59:59.998393Z - 2014-05-01T11:59:59.998393Z | 100.0 Hz, 4320001 samples
CX.PB01..HHE | 2014-04-30T23:59:59.998393Z - 2014-05-01T11:59:59.998393Z | 100.0 Hz, 4320001 samples
CX.PB01..HHN | 2014-04-30T23:59:59.998391Z - 2014-05-01T11:59:59.998391Z | 100.0 Hz, 4320001 samples
CX.PB02..HHZ | 2014-04-30T23:59:59.998393Z - 2014-05-01T11:59:59.998393Z | 100.0 Hz, 4320001 samples
CX.PB02..HHE | 2014-04-30T23:59:59.998393Z - 2014-05-01T11:59:59.998393Z | 100.0 Hz, 4320001 samples
CX.PB02..HHN | 2014-04-30T23:59:59.998393Z - 2014-05-01T11:59:59.998393Z | 100.0 Hz, 4320001 samples
#
processing raw stream data...
detrend...
resampling...
merging...
#
time windows (12) for iteration over continuous waveforms:
[UTCDateTime(2014, 5, 1, 0, 0), UTCDateTime(2014, 5, 1, 1, 0)]
[UTCDateTime(2014, 5, 1, 1, 0), UTCDateTim

## 3. Run phase detection and picking

In [6]:
# model
#
dpp_model = model.Model(verbose=False)
#
print(dpp_model.model_detection['best_model'].summary())
print(dpp_model.model_picking_P['best_model'].summary())
print(dpp_model.model_picking_S['best_model'].summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv1d_1 (Separabl (None, 480, 12)           99        
_________________________________________________________________
activation_1 (Activation)    (None, 480, 12)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 480, 12)           48        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 240, 12)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 240, 12)           0         
_________________________________________________________________
separable_conv1d_2 (Separabl (None, 240, 24)           444       
_________________________________________________________________
activation_2 (Activation)    (None, 240, 24)          

In [7]:
# run phase detection
dpp_model.run_detection(dpp_config, dpp_data, save_dets=False, save_data=False)

#
Calculating predictions for stream: CX.PB01..HH?...
strimming stream: 2, 2
720/720 [==============================] - 29s 40ms/step
3 Trace(s) in Stream:
CX.PB01..HHE | 2014-04-30T23:59:59.998393Z - 2014-05-01T00:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB01..HHN | 2014-04-30T23:59:59.998391Z - 2014-05-01T00:59:59.998391Z | 100.0 Hz, 360001 samples
CX.PB01..HHZ | 2014-04-30T23:59:59.998393Z - 2014-05-01T00:59:59.998393Z | 100.0 Hz, 360001 samples
p_picks = 35, s_picks = 18
#
Calculating predictions for stream: CX.PB02..HH?...
720/720 [==============================] - 33s 46ms/step
3 Trace(s) in Stream:
CX.PB02..HHE | 2014-04-30T23:59:59.998393Z - 2014-05-01T00:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHN | 2014-04-30T23:59:59.998393Z - 2014-05-01T00:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHZ | 2014-04-30T23:59:59.998393Z - 2014-05-01T00:59:59.998393Z | 100.0 Hz, 360001 samples
p_picks = 20, s_picks = 14
#
Calculating predictions for stream: CX.PB01..HH?...
str

720/720 [==============================] - 32s 44ms/step
3 Trace(s) in Stream:
CX.PB01..HHE | 2014-05-01T08:59:59.998393Z - 2014-05-01T09:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB01..HHN | 2014-05-01T08:59:59.998391Z - 2014-05-01T09:59:59.998391Z | 100.0 Hz, 360001 samples
CX.PB01..HHZ | 2014-05-01T08:59:59.998393Z - 2014-05-01T09:59:59.998393Z | 100.0 Hz, 360001 samples
p_picks = 29, s_picks = 16
#
Calculating predictions for stream: CX.PB02..HH?...
720/720 [==============================] - 39s 55ms/step
3 Trace(s) in Stream:
CX.PB02..HHE | 2014-05-01T08:59:59.998393Z - 2014-05-01T09:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHN | 2014-05-01T08:59:59.998393Z - 2014-05-01T09:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHZ | 2014-05-01T08:59:59.998393Z - 2014-05-01T09:59:59.998393Z | 100.0 Hz, 360001 samples
p_picks = 21, s_picks = 12
#
Calculating predictions for stream: CX.PB01..HH?...
strimming stream: 2, 2
720/720 [==============================] - 28s 38ms/step


In [8]:
# run phase picking
dpp_model.run_picking(dpp_config, dpp_data, save_plots=False, save_stats=True, save_picks=False)

['PB01', 'PB02']
#
['1', '2', '3', '4']
detected picks (P, S): 35, 35, 18, 18
1 0 1398902406.46
1 1 1398902530.66
1 2 1398902879.66
1 3 1398902957.56
1 4 1398903192.46
1 5 1398903215.86
1 6 1398903308.76
1 7 1398903347.86
1 8 1398903459.26
1 9 1398903472.66
1 10 1398903540.96
1 11 1398903553.26
1 12 1398903847.26
1 13 1398903877.66
1 14 1398903893.96
1 15 1398903985.36
1 16 1398904198.16
1 17 1398904334.46
1 18 1398904350.66
1 19 1398904448.26
1 20 1398904589.16
1 21 1398904708.56
1 22 1398904780.16
1 23 1398904830.96
1 24 1398904926.76
1 25 1398904945.86
1 26 1398905086.96
1 27 1398905173.36
1 28 1398905196.16
1 29 1398905255.46
1 30 1398905463.16
1 31 1398905524.56
1 32 1398905536.46
1 33 1398905566.56
1 34 1398905957.36
4 0 1
4 1 3
4 2 5
4 3 6
4 4 7
4 5 8
4 6 9
4 7 10
4 8 11
4 9 12
4 10 14
4 11 15
4 12 16
4 13 17
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
3 0 1
3 1 3
3 2 5
3 3 6
3 4 7
3 5 8
3 6 9
3 7 10
3 8 11
3 9 12
3 10 14
3 11 15
selected picks (P, S): 29, 12
#
1

#
['1', '2', '3', '4']
detected picks (P, S): 18, 18, 7, 7
1 0 1398910510.76
1 1 1398910529.16
1 2 1398910763.76
1 3 1398910784.86
1 4 1398910816.56
1 5 1398910831.36
1 6 1398911059.76
1 7 1398911296.36
1 8 1398911316.76
1 9 1398911741.46
1 10 1398911927.46
1 11 1398912051.36
1 12 1398912182.56
1 13 1398912213.76
1 14 1398912224.46
1 15 1398912379.16
1 16 1398912874.36
1 17 1398912944.76
4 0 0
4 1 2
4 2 4
4 3 6
2 0
2 1
2 2
2 3
3 0 0
3 1 2
3 2 4
3 3 6
selected picks (P, S): 15, 4
#
1 1398910510.76 True
2 1398910529.16 False
3 1398910763.76 True
4 1398910784.86 False
5 1398910816.56 True
6 1398910831.36 False
7 1398911059.76 True
8 1398911296.36 True
9 1398911316.76 True
10 1398911741.46 True
11 1398911927.46 True
12 1398912051.36 True
13 1398912182.56 True
14 1398912213.76 True
15 1398912224.46 True
16 1398912379.16 True
17 1398912874.36 True
18 1398912944.76 True
#
1 1398910529.2 True
2 1398910763.8 False
3 1398910784.8 True
4 1398910816.6 False
5 1398910831.4 True
6 1398911316.4 False

['PB01', 'PB02']
#
['1', '2', '3', '4']
detected picks (P, S): 32, 32, 15, 15
1 0 1398924058.16
1 1 1398924096.96
1 2 1398924194.36
1 3 1398924291.46
1 4 1398924305.36
1 5 1398924332.06
1 6 1398924407.06
1 7 1398924421.26
1 8 1398924846.46
1 9 1398925144.06
1 10 1398925311.26
1 11 1398925398.16
1 12 1398925415.16
1 13 1398925642.06
1 14 1398925660.86
1 15 1398925852.56
1 16 1398925893.36
1 17 1398925905.86
1 18 1398926066.26
1 19 1398926248.36
1 20 1398926292.56
1 21 1398926352.26
1 22 1398926628.26
1 23 1398926721.56
1 24 1398926732.56
1 25 1398926744.76
1 26 1398926862.16
1 27 1398926949.66
1 28 1398927056.66
1 29 1398927247.66
1 30 1398927267.06
1 31 1398927584.36
4 0 0
4 1 3
4 2 5
4 3 7
4 4 8
4 5 10
4 6 12
4 7 14
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
3 0 3
3 1 5
3 2 7
3 3 8
3 4 12
3 5 14
selected picks (P, S): 26, 6
#
1 1398924058.16 True
2 1398924096.96 True
3 1398924194.36 True
4 1398924291.46 True
5 1398924305.36 True
6 1398924332.06 True
7 1398924407.06 True
8 1398924421.26 False
9 1

3 4 8
3 5 11
selected picks (P, S): 27, 6
#
1 1398934814.66 True
2 1398934847.16 True
3 1398934976.06 True
4 1398935284.26 True
5 1398935618.66 True
6 1398935704.66 True
7 1398935934.76 True
8 1398936129.66 True
9 1398936377.16 True
10 1398936460.16 True
11 1398936558.96 True
12 1398936811.46 True
13 1398937027.06 True
14 1398937092.56 True
15 1398937107.16 False
16 1398937216.46 True
17 1398937389.56 True
18 1398937409.36 True
19 1398937473.26 True
20 1398937485.86 False
21 1398937609.46 True
22 1398937680.76 True
23 1398937787.16 True
24 1398938102.96 True
25 1398938121.36 True
26 1398938158.66 True
27 1398938182.16 True
28 1398938324.96 True
29 1398938385.26 True
#
1 1398934847.8 False
2 1398934866.3 True
3 1398935618.6 False
4 1398935640.3 True
5 1398935704.5 False
6 1398935726.1 True
7 1398937092.5 False
8 1398937107.2 True
9 1398937238.4 True
10 1398937243.5 False
11 1398937473.6 False
12 1398937485.9 True
13 1398937680.7 False
14 1398937830.9 False
15 1398938121.2 False
16 13989

## 4. Plot predicted phases

In [9]:
# plots
util.plot_predicted_phases(dpp_config, dpp_data, dpp_model)
# util.plot_predicted_phases(dpp_config, dpp_data, dpp_model, plot_probs=['P','S'], shift_probs=True)

creating plots...
1 PB01 Z 2014-04-30T23:59:59.998393Z 2014-05-01T00:59:59.998393Z
1 PB01 E 2014-04-30T23:59:59.998393Z 2014-05-01T00:59:59.998393Z
1 PB02 Z 2014-04-30T23:59:59.998393Z 2014-05-01T00:59:59.998393Z
1 PB02 E 2014-04-30T23:59:59.998393Z 2014-05-01T00:59:59.998393Z
2 PB01 Z 2014-05-01T00:59:59.998393Z 2014-05-01T01:59:59.998393Z
2 PB01 E 2014-05-01T00:59:59.998393Z 2014-05-01T01:59:59.998393Z
2 PB02 Z 2014-05-01T00:59:59.998393Z 2014-05-01T01:59:59.998393Z
2 PB02 E 2014-05-01T00:59:59.998393Z 2014-05-01T01:59:59.998393Z
3 PB01 Z 2014-05-01T01:59:59.998393Z 2014-05-01T02:59:59.998393Z
3 PB01 E 2014-05-01T01:59:59.998393Z 2014-05-01T02:59:59.998393Z
3 PB02 Z 2014-05-01T01:59:59.998393Z 2014-05-01T02:59:59.998393Z
3 PB02 E 2014-05-01T01:59:59.998393Z 2014-05-01T02:59:59.998393Z
4 PB01 Z 2014-05-01T02:59:59.998393Z 2014-05-01T03:59:59.998393Z
4 PB01 E 2014-05-01T02:59:59.998393Z 2014-05-01T03:59:59.998393Z
4 PB02 Z 2014-05-01T02:59:59.998393Z 2014-05-01T03:59:59.998393Z
4 PB02 